In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import nibabel as nib
import SimpleITK as sitk
import tensorflow.compat.v1 as tf
import tensorflow.keras.backend as K

from numpy import mean
from numpy import std
from tqdm import tqdm
from visualization import all_axes, og_slice, rs_slice
from skimage.transform import resize
from skimage.io import imshow
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from time import time

from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, PReLU, MaxPool3D
#from keras.layers.pooling import MaxPooling3D
from keras.layers.merge import concatenate, add
from tensorflow.keras.layers import Concatenate, Add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.nn import conv3d_transpose

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
torch.cuda.empty_cache()

## Crop and Load Training/Mask Images

In [ ]:
def get_np_volume_from_sitk(sitk_img):
    trans = (2, 1, 0)
    px_spacing = sitk_img.GetSpacing()
    img_position = sitk_img.GetOrigin()
    np_img = sitk.GetArrayFromImage(sitk_img)
    np_img = np.transpose(np_img, trans)
    return np_img, px_spacing, img_position


ids = []
img_count = 0

dir_path = "./hecktor_train/hecktor_nii/"
bb_path = './hecktor_train/bbox.csv'
bb_dict = pd.read_csv(bb_path).set_index('PatientID')

id_path = os.listdir(dir_path)

for _id in id_path:
    if str(_id) != ".DS_Store":
        ids.append(_id)

ids=ids[:55]        
        
X = np.zeros((len(ids), 96, 96, 96))
y = np.zeros((len(ids), 96, 96, 96))

for _id in tqdm(ids):
    ct_path = dir_path + _id + "/" + _id + "_ct.nii.gz"
    gt_path = dir_path + _id + "/" + _id + "_ct_gtvt.nii.gz"

    ct_img, spacing, origin = get_np_volume_from_sitk(sitk.ReadImage(ct_path))
    gt_img, spacing, origin = get_np_volume_from_sitk(sitk.ReadImage(gt_path))

    bb = np.round((np.asarray([
    bb_dict.loc[_id, 'x1'],
    bb_dict.loc[_id, 'y1'],
    bb_dict.loc[_id, 'z1'],
    bb_dict.loc[_id, 'x2'],
    bb_dict.loc[_id, 'y2'],
    bb_dict.loc[_id, 'z2']
    ]) - np.tile(origin, 2)) / np.tile(spacing, 2)).astype(int) 
    
    ct_img = ct_img[bb[0]:bb[3], bb[1]:bb[4], bb[2]:bb[5]]
    gt_img = gt_img[bb[0]:bb[3], bb[1]:bb[4], bb[2]:bb[5]]
    
    ct_img = resize(ct_img, (96, 96, 96), mode="constant", preserve_range=True)
    gt_img = resize(gt_img, (96, 96, 96), mode="constant", preserve_range=True)

    X[img_count, :, :, :] = ct_img
    y[img_count, :, :, :] = gt_img

    img_count += 1

## Normalize

In [ ]:
# Normalize images
X = (X - X.min()) / (X.max() - X.min())

In [ ]:
y[y>0] = 1

In [ ]:
X=K.expand_dims(X,-1)
y=K.expand_dims(y,-1)

In [ ]:
X = X[0:10,:,:,:,:]
y = y[0:10,:,:,:,:]

In [ ]:
y.shape

## DSC Functions

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    sum_y_true = K.sum(y_true_f)
    sum_y_pred = K.sum(y_pred_f)
    
    return ((2. * intersection + 1.) / (sum_y_true + sum_y_pred + 1.))


def dice_coef_mod(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    dice_coefficient_list = []
    thresholds = [.5]
    
    for threshold in thresholds:
        y_true_binary = tf.where(y_true_f > threshold, tf.ones_like(y_true_f), tf.zeros_like(y_true_f))
        y_pred_binary = tf.where(y_pred_f > threshold, tf.ones_like(y_pred_f), tf.zeros_like(y_pred_f))
        intersection = K.sum(y_true_binary * y_pred_binary)
        dice_coefficient = (2. * intersection + smooth) / (K.sum(y_true_binary) + K.sum(y_pred_binary) + smooth)
        dice_coefficient_list.append(dice_coefficient)
    
    total = 0
    
    for i in range(len(dice_coefficient_list)):
        total += dice_coefficient_list[i]
        
    average = total / len(dice_coefficient_list)
    
    return average


def dice_coef_loss(y_true, y_pred):
    return (1-dice_coef(y_true, y_pred))

## V-Net Model

In [ ]:
def v_net():
    input_img = Input((96, 96, 96, 1))

    # Layer 1
    conv1 = Conv3D(16, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (input_img)
    conv1 = PReLU() (conv1)
    input1 = Concatenate(axis=-1) (16 * [input_img])
    add1 = Add() ([input1, conv1])
    down1 = Conv3D(32, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add1)
    down1 = PReLU() (down1)

    # Layer 2
    conv2 = Conv3D(32, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (down1)
    conv2 = PReLU() (conv2)
    conv2 = Conv3D(32, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv2)
    conv2 = PReLU() (conv2)
    add2 = Add() ([conv2, down1])
    down2 = Conv3D(64, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add2)
    down2 = PReLU() (down2)

    # Layer 3
    conv3 = Conv3D(64, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (down2)
    conv3 = PReLU() (conv3)
    conv3 = Conv3D(64, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv3)
    conv3 = PReLU() (conv3)
    conv3 = Conv3D(64, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv3)
    conv3 = PReLU() (conv3)
    add3 = Add() ([conv3, down2])
    down3 = Conv3D(128, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add3)
    down3 = PReLU() (down3)

    # Layer 4
    conv4 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (down3)
    conv4 = PReLU() (conv4)
    conv4 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv4)
    conv4 = PReLU() (conv4)
    conv4 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv4)
    conv4 = PReLU() (conv4)
    add4 = Add() ([conv4, down3])
    down4 = Conv3D(256, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add4)
    down4 = PReLU() (down4)

    # Layer 5
    conv5 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (down4)
    conv5 = PReLU() (conv5)
    conv5 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv5)
    conv5 = PReLU() (conv5)
    conv5 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv5)
    conv5 = PReLU() (conv5)
    add5 = Add() ([conv5, down4])
    up5 = Conv3DTranspose(128, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add5)
    up5 = PReLU() (up5)

    # Layer 6
    skipcon6 = Concatenate(axis=4) ([up5, add4])
    conv6 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (skipcon6)
    conv6 = PReLU() (conv6)
    conv6 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv6)
    conv6 = PReLU() (conv6)
    conv6 = Conv3D(256, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv6)
    conv6 = PReLU() (conv6)
    add6 = Add() ([conv6, skipcon6])
    up6 = Conv3DTranspose(64, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add6)
    up6 = PReLU() (up6)

    # Layer 7
    skipcon7 = Concatenate(axis=4) ([up6, add3])
    conv7 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (skipcon7)
    conv7 = PReLU() (conv7)
    conv7 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv7)
    conv7 = PReLU() (conv7)
    conv7 = Conv3D(128, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv7)
    conv7 = PReLU() (conv7)
    add7 = Add() ([conv7, skipcon7])
    up7 = Conv3DTranspose(32, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add7)
    up7 = PReLU() (up7)

    # Layer 8
    skipcon8 = Concatenate(axis=4) ([up7, add2])
    conv8 = Conv3D(64, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (skipcon8)
    conv8 = PReLU() (conv8)
    conv8 = Conv3D(64, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (conv8)
    conv8 = PReLU() (conv8)
    add8 = Add() ([conv8, skipcon8])
    up8 = Conv3DTranspose(16, kernel_size=2, strides=2, kernel_initializer="he_normal", activation='relu') (add8)
    up8 = PReLU() (up8)

    # Layer 9
    skipcon9 = Concatenate(axis=4) ([up8, add1])
    conv9 = Conv3D(32, kernel_size=5, padding="same", kernel_initializer="he_normal", activation='relu') (skipcon9)
    conv9 = PReLU() (conv9)
    add9 = Add() ([conv9, skipcon9])
    conv9 = Conv3D(1, kernel_size=1, padding="same", kernel_initializer="he_normal", activation='relu') (add9)
    conv9 = PReLU() (conv9)

    sigmoid = Conv3D(1, kernel_size=1, padding="same", kernel_initializer="he_normal", activation='sigmoid') (conv9)

    model = Model(inputs=input_img, outputs=sigmoid)
        
    return model

In [ ]:
model=v_net()
model.compile(optimizer=Adam(lr=1e-4), loss=tf.keras.losses.MeanAbsoluteError(), metrics=[dice_coef, dice_coef_mod, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
history = model.fit(y, y, batch_size=1, epochs=50, validation_split=0.3)

In [ ]:
def conv_block(input_layer, filters, kernel_size):
    c = Conv3D(filters, kernel_size = (kernel_size, kernel_size, kernel_size), padding = 'same') (input_layer)
    c = BatchNormalization() (c)
    c = Activation('relu') (c)
    c = Conv3D(filters, kernel_size = (kernel_size, kernel_size, kernel_size), padding = 'same') (c)
    c = BatchNormalization() (c)
    c = Activation('relu') (c)
    c = add([input_layer, c])
    
    return c


def v_net(input_img, filters = 8, dropout = 0.2):
    c1 = Conv3D(filters, kernel_size = (5, 5, 5), padding = 'same') (input_img)
    
    c2 = Conv3D(filters*2, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (c1)
    c2 = conv_block(c2, filters*2, 5)
    
    c3 = Conv3D(filters*4, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (c2)
    c3 = Dropout(dropout) (c3)
    c3 = conv_block(c3, filters*4, 5)
    
    c4 = Conv3D(filters*8, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (c3)
    c4 = Dropout(dropout) (c4)
    c4 = conv_block(c4, filters*8, 5)
    
    c5 = Conv3D(filters*16, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (c4)
    c5 = Dropout(dropout) (c5)
    c5 = conv_block(c5, filters*16, 5)
    
    u6 = Conv3DTranspose(filters*8, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (c5)
    u6 = concatenate([u6, c4])
    u6 = conv_block(u6, filters*16, 5)
    u6 = Dropout(dropout) (u6)
    
    u7 = Conv3DTranspose(filters*4, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (u6)
    u7 = concatenate([u7, c3])
    u7 = conv_block(u7, filters*8, 5)
    u7 = Dropout(dropout) (u7) 
    
    u8 = Conv3DTranspose(filters*2, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (u7)
    u8 = concatenate([u8, c2])
    u8 = conv_block(u8, filters*4, 5)
    u8 = Dropout(dropout) (u8) 
    
    u9 = Conv3DTranspose(filters, kernel_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (u8)
    u9 = concatenate([u9, c1])
    
    c9 = Conv3D(filters*2, kernel_size = (5, 5, 5), padding = 'same') (u9)
    c9 = Dropout(dropout) (c9)
    c9 = add([c9, u9])
    
    outputs = Conv3D(1, kernel_size = (1, 1, 1), activation = 'softmax') (c9)
    
    model = Model(inputs = input_img, outputs = outputs)
    
    return model

In [ ]:
model.evaluate(X[1:2,:,:,:,:],y[1:2,:,:,:,:])

In [ ]:
#y_temp[0:1,:,:,:,:].shape

In [ ]:
plt.imshow(y[1,40,:,:,0])

In [ ]:
plt.imshow(model.predict(X[1:2,:,:,:,0])[0,60,:,:,0])

In [ ]:
#np.amax(model.predict(X[0:1,:,:,:,:])[0,80,:,:,0])

In [ ]:
#np.amax(X[0:1,80,:,:,0])

In [ ]:
#model.summary(line_length=150)

In [ ]:
#random_indices = np.random.choice(201, size=10, replace=False)
#random_indices

In [ ]:
#X = X[random_indices, :, :, :]

In [ ]:
#y = y[random_indices, :, :, :]

## Visualize Images

In [ ]:
#all_axes("CHGJ007", -30, 100, -100)

In [ ]:
#og_slice("CHGJ007", 46)

In [ ]:
#rs_slice("CHGJ007", 32, 128, 128, 64)

In [ ]:
'''def conv_block(input_layer, filters, kernel_size):
    c = Conv3D(filters, kernel_size = (kernel_size, kernel_size, kernel_size), padding = 'same') (input_layer)
    c = BatchNormalization() (c)
    c = Activation('relu') (c)
    c = Conv3D(filters, kernel_size = (kernel_size, kernel_size, kernel_size), padding = 'same') (c)
    c = BatchNormalization() (c)
    c = Activation('relu') (c)
    
    return c


def v_net(input_img, n_filters = 8, dropout = 0.2):
    c1 = conv_block(input_img,n_filters,3)
    p1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv_block(p1,n_filters*2,3);
    p2 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv_block(p2,n_filters*4,3);
    p3 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv_block(p3,n_filters*8,3);
    p4 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv_block(p4,n_filters*16,3);
    
    u6 = Conv3DTranspose(n_filters*8, (3,3,3), strides=(2, 2, 2), padding='same')(c5);
    u6 = concatenate([u6,c4]);
    
    c6 = conv_block(u6,n_filters*8,3)
    c6 = Dropout(dropout)(c6)
    
    u7 = Conv3DTranspose(n_filters*4,(3,3,3),strides = (2,2,2) , padding= 'same')(c6);
    u7 = concatenate([u7,c3]);
    c7 = conv_block(u7,n_filters*4,3)
    c7 = Dropout(dropout)(c7)
    
    u8 = Conv3DTranspose(n_filters*2,(3,3,3),strides = (2,2,2) , padding='same')(c7);
    u8 = concatenate([u8,c2]);
    
    c8 = conv_block(u8,n_filters*2,3)
    c8 = Dropout(dropout)(c8)
    
    u9 = Conv3DTranspose(n_filters,(3,3,3),strides = (2,2,2) , padding='same')(c8);
    u9 = concatenate([u9,c1]);
    c9 = conv_block(u9,n_filters,3)
    
    outputs = Conv3D(1, (1,1,1), activation='softmax')(c9)
    
    print("!!!!!!!!!!!!!!!!!!!")
    print(outputs.shape)
    
    model = Model(inputs=input_img, outputs=outputs)
    
    return model'''

In [ ]:
#np.set_printoptions(threshold=np.inf)
#print(X[200][70])

In [ ]:
#print(history.history.keys())

In [ ]:
'''per_dic = {}

per_dic['loss']=history.history['loss']    
per_dic['dice_coef']=history.history['dice_coef']    
per_dic['dice_coef_mod']=history.history['dice_coef_mod'] 
per_dic['precision']=history.history['precision_3'] 
per_dic['recall']=history.history['recall_3'] 
per_dic['val_loss']=history.history['val_loss']
per_dic['val_dice_coef']=history.history['val_dice_coef']
per_dic['val_dice_coef_mod']=history.history['val_dice_coef_mod']
per_dic['val_precision']=history.history['val_precision_3']
per_dic['val_recall']=history.history['val_recall_3']'''

In [ ]:
#p = pd.DataFrame.from_dict(per_dic)

#with pd.ExcelWriter('MAE peformance.xlsx') as writer:  
    #p.to_excel(writer, sheet_name='performance')

In [ ]:
#preds = model.predict(X)
#score = model.evaluate(X, y, verbose=0, batch_size=2)

In [ ]:
'''kf = KFold(n_splits=5)
performance = {}
k = 1

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    
    X_train, X_test = X[train_index, :, :, :], X[test_index, :, :, :]
    y_train, y_test = y[train_index, :, :, :], y[test_index, :, :, :]
    
    model = v_net()
    start = time()
    history = model.fit(X_train, y_train, batch_size=1, epochs=20 , validation_split=0.3)
    elapsed = (time()-start)/60
    score = model.evaluate(X_test, y_test, verbose=0, batch_size=1)
    
    per_dic = {}
    per_dic['Testdiceloss']=history.history['val_loss']    
    per_dic['Testdsc']=score[1]   
    per_dic['Testprec']=score[2]
    per_dic['Testrec']=score[3]       
    per_dic['elapsed_minutes']=elapsed 
    performance[k]=per_dic
    
    k += 1'''

In [ ]:
#performance

In [ ]:
'''p1 = pd.DataFrame.from_dict(performance[1])
p2 = pd.DataFrame.from_dict(performance[2])
p3 = pd.DataFrame.from_dict(performance[3])
p4 = pd.DataFrame.from_dict(performance[4])
p5 = pd.DataFrame.from_dict(performance[5])'''

In [ ]:
'''with pd.ExcelWriter('model peformance.xlsx') as writer:  
    p1.to_excel(writer, sheet_name='k1')
    p2.to_excel(writer, sheet_name='k2')
    p3.to_excel(writer, sheet_name='k3')
    p4.to_excel(writer, sheet_name='k4')
    p5.to_excel(writer, sheet_name='k5')'''

In [9]:
rates=[1,0.01,0.001,0.0001,0.00001]
for exp in range(-5, 0):
    print(10**exp)

1e-05
0.0001
0.001
0.01
0.1


In [18]:
patient_index=[]
for i in range(0, 201):
    for j in range(0, 96):
        patient_index.append(i)
patient_index

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [24]:
import random
random_list = np.random.randint(10,size=201)
random_list

array([6, 7, 0, 9, 9, 4, 0, 8, 6, 4, 8, 9, 8, 2, 8, 1, 9, 5, 9, 9, 1, 2,
       8, 3, 4, 8, 9, 7, 7, 8, 2, 3, 9, 9, 5, 2, 4, 2, 0, 1, 4, 2, 6, 1,
       4, 3, 9, 0, 4, 5, 8, 2, 5, 0, 1, 6, 0, 8, 1, 5, 8, 7, 8, 9, 9, 0,
       5, 4, 8, 7, 4, 2, 7, 8, 0, 2, 5, 2, 4, 6, 1, 2, 7, 6, 7, 1, 7, 7,
       2, 1, 6, 0, 6, 5, 9, 0, 3, 3, 4, 3, 2, 0, 8, 1, 9, 4, 5, 6, 7, 7,
       4, 4, 8, 9, 2, 1, 1, 7, 1, 8, 7, 1, 4, 0, 1, 3, 1, 7, 0, 1, 3, 1,
       0, 3, 5, 9, 3, 7, 9, 8, 9, 6, 4, 5, 0, 4, 5, 1, 7, 5, 8, 9, 4, 9,
       6, 9, 6, 2, 3, 9, 2, 3, 8, 0, 0, 0, 5, 2, 2, 7, 0, 8, 7, 9, 5, 3,
       6, 5, 7, 8, 8, 2, 9, 9, 8, 3, 1, 0, 8, 5, 6, 1, 9, 0, 2, 8, 9, 8,
       0, 5, 4])